# Reappraisal Training on PyTorch Lightning

## Setup
- When running on Google Colab, mount Google Drive to access scripts.
- `cd` into the project root and install dependencies.

In [ ]:
%load_ext autoreload
# from google.colab import drive
# drive.mount('/content/drive')
# %cd {root_dir}

%pip install pytorch-lightning "ray[tune]" wandb transformers datasets nltk


import nltk
nltk.download('punkt')

ROOT_DIR = '/home/ec2-user/SageMaker/reappraisal-model/'

In [ ]:
import torch

# Define constants
STRAT = 'far'
BATCH_SIZE = 64 if not torch.cuda.is_available() else 16
TEST_BATCH_SIZE = 64
NUM_FOLDS = 5


In [ ]:
from reappraisalmodel.ldhdata import LDHDataModule
ldhdata = LDHDataModule(data_dir=ROOT_DIR, batch_size=BATCH_SIZE, strat=STRAT, kfolds=NUM_FOLDS)

In [ ]:
from reappraisalmodel.trainers import kfold_train
results = kfold_train(NUM_FOLDS, ldhdata, strat=STRAT, max_epochs=20)

In [ ]:
%autoreload
from reappraisalmodel.trainers import run_tune
run_tune(ldhdata)

In [ ]:
%autoreload
import torch
from reappraisalmodel.lightningreapp import LightningReapp

default_config = {
    'lr': 1e-3,
    'hidden_layer_size': 50,
    'strat': STRAT
}
model = LightningReapp.load_from_checkpoint('/Users/danielpham/Google Drive/ldh/lightning_logs_obj_0223/version_2/checkpoints/epoch=1-step=337.ckpt',
)

model.eval()
model

In [ ]:
from pathlib import Path

os.getcwd()